<a href="https://colab.research.google.com/github/adipai/data-decent/blob/main/src/automation/automation_Eclipse_JDT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pmlb

In [2]:
!pip install sdv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 937.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.8/169.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of sdmetrics to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 19.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtim

In [15]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/imbalance_defects_prediction/
   creating: data/project_health/
  inflating: data/README.md          
   creating: data/JavaScript_Vulnerability/
   creating: data/Bug_Reports/
   creating: data/Vulnerable_Files/
   creating: data/defects_prediction/
   creating: data/imbalance_defects_prediction/7_CK_NET_PROC/
   creating: data/imbalance_defects_prediction/2_NET/
   creating: data/imbalance_defects_prediction/4_CK_NET/
   creating: data/imbalance_defects_prediction/3_PROC/
   creating: data/imbalance_defects_prediction/6_NET_PROC/
   creating: data/imbalance_defects_prediction/1_CK/
   creating: data/imbalance_defects_prediction/5_CK_PROC/
   creating: data/project_health/monthly_closed_PRs_2mo/
   creating: data/project_health/monthly_commits_2mo/
   creating: data/project_health/monthly_open_PRs_2mo/
   creating: data/project_health/monthly_closed_issues_2mo/
   creating: data/project_health/monthly_commits_12mo/
   creating: d

In [4]:
# All imports here
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from pmlb import fetch_data
import matplotlib.pyplot as plt
import seaborn as sns
import random
import time
import math
import warnings

from scipy.io import arff
from sdv.datasets.local import load_csvs
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import RandomOverSampler
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

In [5]:
warnings.filterwarnings('ignore')

## Data preprocessing

In [6]:
def preprocess_data_train(X_train):
    # Count missing values before handling missing data
    missing_before = np.isnan(X_train).sum()

    # Handle missing data
    imputer = SimpleImputer(strategy='mean')
    X_train = imputer.fit_transform(X_train)

    # Count missing values after handling missing data
    missing_after = np.isnan(X_train).sum()

    # Normalize numeric columns
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    return X_train, scaler, imputer

def preprocess_data_test(X_test, scaler, imputer):
    # Count missing values before handling missing data
    missing_before = np.isnan(X_test).sum()

    # Handle missing data
    X_test = imputer.transform(X_test)

    # Count missing values after handling missing data
    missing_after = np.isnan(X_test).sum()

    # Normalize numeric columns
    X_test = scaler.transform(X_test)

    return X_test

## Experiments

### Dataset: Eclipse JDT

In [105]:
project = "Defect_Eclipse_JDT_Core"
fname = "_".join(project.split("_")[1:])
data_path = f"data/imbalance_defects_prediction/7_CK_NET_PROC/input/{fname}--CK_NET_PROC.arff"
data = arff.loadarff(data_path)
df = pd.DataFrame(data[0])
df['isBug'] = df['isBug'].astype('str')
d = {'YES': 1, 'NO': 0}  # Remove the byte string prefix 'b'
df['isBug'] = df['isBug'].map(d).fillna(df['isBug'])
print(df['isBug'])
print("before drop duplicates", df.shape[0])
df = df.drop_duplicates()
df.reset_index(inplace=True, drop=True)
print("after drop duplicates", df.shape[0])

df.describe()

0      0
1      0
2      0
3      0
4      0
      ..
992    0
993    0
994    0
995    0
996    0
Name: isBug, Length: 997, dtype: int64
before drop duplicates 997
after drop duplicates 997


,wmc,dit,rfc,noc,cbo,lcom,loc,revision_num,author_num,linesadd_sum,...,InFreeClo,OutValClo,InValClo,OutRecipClo,InRecipClo,OutdwReach,IndwReach,nOutdwReach,nIndwReach,isBug
count,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000,...,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000,997.000000
mean,58.384152,2.727182,76.874624,0.712136,12.216650,364.727182,224.729188,45.618857,5.793380,1209.460381,...,0.003507,0.577920,0.577920,0.173663,0.173663,173.548682,173.548714,0.174071,0.174071,0.206620
std,135.722660,1.721525,180.978591,2.154752,17.815915,3230.074059,555.700530,60.995862,2.570187,3921.650184,...,0.002629,0.388013,0.202109,0.106940,0.089645,106.878591,89.463820,0.107200,0.089733,0.405084
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.001003,0.000000,0.000000,0.001003,0.001003,1.000000,1.000000,0.001003,0.001003,0.000000
25%,8.000000,1.000000,12.000000,0.000000,3.000000,6.000000,28.000000,13.000000,4.000000,81.000000,...,0.002014,0.257659,0.502486,0.092025,0.118853,91.916748,118.883202,0.092193,0.119241,0.000000
50%,20.000000,2.000000,30.000000,0.000000,7.000000,28.000000,75.000000,30.000000,6.000000,276.000000,...,0.002016,0.951735,0.502911,0.162254,0.150396,162.562225,150.299881,0.163051,0.150752,0.000000
75%,50.000000,4.000000,70.000000,0.000000,14.000000,91.000000,192.000000,52.000000,7.000000,833.000000,...,0.007888,0.953893,0.873717,0.267023,0.225866,266.911713,225.466293,0.267715,0.226145,0.000000
max,1680.000000,8.000000,2603.000000,26.000000,156.000000,81003.000000,7341.000000,709.000000,15.000000,65571.000000,...,0.008033,0.970087,0.876018,0.438462,0.534813,437.664520,533.550598,0.438981,0.535156,1.000000


## Training and testing using ML models

In [106]:
# Generic function to test synthetic data using LR, SVM, DT

def evaluate_models(X_train, X_test, y_train, y_test, random_state=42):

    # Initialize classifiers
    classifiers = {
        "Logistic Regression": LogisticRegression(random_state=random_state),
        "SVM": SVC(random_state=random_state),
        "Decision Tree": DecisionTreeClassifier(random_state=random_state)
    }

    # Results dictionary to store evaluation metrics
    results = {}

    # Iterate over classifiers
    for name, clf in classifiers.items():
        # Fit classifier
        clf.fit(X_train, y_train)

        # Predictions
        y_pred = clf.predict(X_test)

        # Evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # AUC-ROC
        if hasattr(clf, "predict_proba"):
            y_prob = clf.predict_proba(X_test)[:,1]
        else:
            y_prob = clf.decision_function(X_test)
        fpr, tpr, thresholds = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)

        # Store results
        results[name] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1,
            "ROC AUC": roc_auc,
            "Confusion Matrix": cm
        }

        # Plot AUC-ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{name} - AUC-ROC Curve')
        plt.legend(loc='lower right')
        plt.savefig(f'{name}_auc_roc_curve.png', dpi=300)
        plt.close()

        # Plot confusion matrix
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'{name} - Confusion Matrix')
        plt.savefig(f'{name}_confusion_matrix.png', dpi=300)
        plt.close()

    return results

In [107]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [108]:
results = evaluate_models(X_train, X_test, y_train, y_test)

In [109]:
print(results)

{'Logistic Regression': {'Accuracy': 0.84, 'Precision': 0.7777777777777778, 'Recall': 0.44680851063829785, 'F1 Score': 0.5675675675675675, 'ROC AUC': 0.8149075232930052, 'Confusion Matrix': array([[147,   6],
       [ 26,  21]])}, 'SVM': {'Accuracy': 0.82, 'Precision': 0.7894736842105263, 'Recall': 0.3191489361702128, 'F1 Score': 0.4545454545454545, 'ROC AUC': 0.7744402725629259, 'Confusion Matrix': array([[149,   4],
       [ 32,  15]])}, 'Decision Tree': {'Accuracy': 0.78, 'Precision': 0.5306122448979592, 'Recall': 0.5531914893617021, 'F1 Score': 0.5416666666666667, 'ROC AUC': 0.7014323459880406, 'Confusion Matrix': array([[130,  23],
       [ 21,  26]])}}


# RRP Over-sampling

In [110]:
def cluster(candidates, enough, res):
    if len(candidates) < enough:
        res.append(candidates)
        return res

    east, west, east_items, west_items = split(candidates)
    res = cluster(east_items, enough, res)
    res = cluster(west_items, enough, res)

    return res


def split(candidates):
    pivot = random.choice(candidates)
    east_pivot = find_farest(pivot, candidates)
    west_pivot = find_farest(east_pivot, candidates)
    c = cal_distance(east_pivot, west_pivot)

    if c == 0:
        east_items = candidates[:len(candidates)//2]
        west_items = candidates[len(candidates)//2:]
        return east_pivot, west_pivot, east_items, west_items

    all_distance = []
    for candidate in candidates:
        a = cal_distance(candidate, west_pivot)
        b = cal_distance(candidate, east_pivot)
        d = (a ** 2 + c ** 2 - b ** 2) / (2 * c)
        all_distance.append((d, candidate))

    all_distance.sort(key=lambda x: x[0])
    sorted_candidates = [item[1] for item in all_distance]
    east_items = sorted_candidates[:len(sorted_candidates)//2]
    west_items = sorted_candidates[len(sorted_candidates)//2:]

    return east_pivot, west_pivot, east_items, west_items


def find_farest(pivot, candidates):
    max_d = 0
    most_point = pivot

    for candidate in candidates:
        cur_d = cal_distance(pivot, candidate)
        if  cur_d > max_d:
            max_d = cur_d
            most_point = candidate

    return most_point


def cal_distance(p1, p2):
    return math.sqrt(sum([(v1 - v2) ** 2 for v1, v2 in zip(p1[:-1], p2[:-1])]))


def process_mixed_cluster(cluster):
    """
    in DE operation, use current-to-best to mutate the candidates
    v_i = x_i + F * (x_b - x_i) + F_min * (x_r1 - x_r2)
    """
    DE_params = {"F": 0.8, "Fmin": 0.1, "CR": 1.0}
    pos_point = [(idx, item) for idx, item in enumerate(cluster) if item[-1] == 1]
    neg_point = [(idx, item) for idx, item in enumerate(cluster) if item[-1] == 0]

    candidate_l = []
    if len(pos_point) == 1:
        # only 1 pos point in cluster, then mutate all neg points toward to the pos point
        xb = pos_point[0][1]
        R = random.choice(range(len(xb)-1))

        for _, xi in neg_point:
            new_candidate = []
            for i in range(len(xi)-1):
                ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                if ri < DE_params["CR"] or i == R:
                    new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]))
                else:
                    new_candidate.append(xi[i])

            new_candidate.append(1)
            candidate_l.append(np.array(new_candidate))
    else:
        # more than 2 pos points in cluster, then randomly pick 3 points, first 1 is current point, and
        # another two are support points
        for idx1, xb in pos_point:
            R = random.choice(range(len(xb)-1))
            for idx2, xi in enumerate(cluster):
                if not idx1 == idx2:
                    available_points = []
                    for idx3, p in enumerate(cluster):
                        if not idx3 == idx2 and not idx3 == idx1:
                            available_points.append(p)
                    [xr1, xr2] = random.sample(available_points, 2)

                    new_candidate = []
                    # if xr1 and xr2 all negative class, then just use xi
                    if xr1[-1] == 0 and xr2[-1] == 0:
                        for i in range(len(xi)-1):
                            ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                            if ri < DE_params["CR"] or i == R:
                                new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]))
                            else:
                                new_candidate.append(xi[i])
                    else:
                        for i in range(len(xi)-1):
                            ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                            if ri < DE_params["CR"] or i == R:
                                if xr1[-1] == 1:
                                    new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]) + DE_params["Fmin"] * (xr1[i] - xr2[i]))
                                else:
                                    new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]) + DE_params["Fmin"] * (xr2[i] - xr1[i]))
                            else:
                                new_candidate.append(xi[i])

                    new_candidate.append(1)
                    candidate_l.append(np.array(new_candidate))

    return candidate_l


def process_positive_cluster(cluster):
    """
    in DE operation, use best to mutate the candidates
    v_i = x_b + F * (x_r1 - x_r2)
    """
    DE_params = {"F": 0.8, "CR": 1.0}
    pos_point = [item for item in cluster if item[-1] == 1]
    candidate_l = []

    for idx1 in range(len(pos_point)-2):
        for idx2 in range(idx1+1, len(pos_point)-1):
            for idx3 in range(idx2+1, len(pos_point)):
                [xb, xr1, xr2] = random.sample([idx1, idx2, idx3], 3)
                xb, xr1, xr2 = pos_point[xb], pos_point[xr1], pos_point[xr2]

                R = random.choice(range(len(xb)-1))
                new_candidate = []

                for i in range(len(xb)-1):
                    ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                    if ri < DE_params["CR"] or i == R:
                        new_candidate.append(xb[i] + DE_params["F"] * (xr1[i] - xr2[i]))
                    else:
                        new_candidate.append(xb[i])

                new_candidate.append(1)
                candidate_l.append(np.array(new_candidate))

    return candidate_l


def process_mixed_cluster_extra(cluster):
    """
    in DE operation, use current-to-best-extra to mutate the candidates
    v_i = x_b + F * (x_r1 - x_r2) + F_ex * (x_r3 - x_r4)
    """
    DE_params = {"F": 0.8, "CR": 1.0, "F_xc": 0.1}
    pos_point = [item for item in cluster if item[-1] == 1]

    candidate_l = []
    for xb in pos_point:
        R = random.choice(range(len(xb)-1))

        for xi in cluster:
            if not np.array_equal(xb, xi):
                available_points = []
                for p in cluster:
                    if not np.array_equal(p, xi) and not np.array_equal(p, xb):
                        available_points.append(p)

                for _ in range(20):
                    [xr1, xr2, xr3, xr4] = random.sample(available_points, 4)

                    new_candidate = []

                    for i in range(len(xi)-1):
                        ri = np.random.uniform(low=0.0, high=1.0, size=1)[0]

                        if ri < DE_params["CR"] or i == R:
                            new_candidate.append(xi[i] + DE_params["F"] * (xb[i] - xi[i]) + DE_params["F_xc"] * (xr1[i] - xr2[i]) + DE_params["F_xc"] * (xr3[i] - xr4[i]))
                        else:
                            new_candidate.append(xi[i])

                    new_candidate.append(1)
                    candidate_l.append(new_candidate)

    return candidate_l


def RandomProjectionOversampling(X_train, y_train, threshold):
    train_df = pd.concat([X_train, y_train], axis=1)
    train_df.reset_index(inplace=True, drop=True)
    tar = y_train.name
    X_train[tar] = y_train
    X_train.reset_index(inplace=True, drop=True)
    col_names = X_train.columns

    n_data_to_generate = X_train[tar].value_counts()[0] - X_train[tar].value_counts()[1]
    X_train = X_train.to_numpy()
    start_time = time.time()
    res = cluster(X_train, threshold, [])

    new_data_negative_cluster = []
    new_data_positive_cluster = []
    for c in res:
        if sum([item[-1] for item in c]) > len(c)//2:
            cur_new_data = process_positive_cluster(c)
            new_data_positive_cluster += cur_new_data
        else:
            cur_new_data = process_mixed_cluster(c)
            new_data_negative_cluster += cur_new_data

    rt = time.time() - start_time
    print(len(new_data_negative_cluster), n_data_to_generate - len(new_data_positive_cluster), n_data_to_generate, len(new_data_positive_cluster))
    if len(new_data_negative_cluster) >= n_data_to_generate - len(new_data_positive_cluster):

        if(n_data_to_generate > len(new_data_positive_cluster)):
            new_data = new_data_positive_cluster + random.sample(new_data_negative_cluster,
                                                            n_data_to_generate - len(new_data_positive_cluster))
        else:
            new_data = new_data_positive_cluster
    else:
        extra_data = []
        for c in res:
            cur_extra_data = process_mixed_cluster_extra(c)
            extra_data += cur_extra_data

        rest_data_to_generate = n_data_to_generate - len(new_data_positive_cluster) - len(new_data_negative_cluster)
        new_data = new_data_negative_cluster + new_data_positive_cluster + random.sample(extra_data, rest_data_to_generate)

    new_data_df = pd.DataFrame(np.array(new_data), columns=col_names)

    return rt, new_data_df, train_df

# SDV - Oversampling

In [111]:
def do_sdv(X_train, y_train):
  train_df = pd.concat([X_train, y_train], axis=1)
  class_counts = y_train.value_counts()

  # Find minority class label
  minority_class_label = class_counts.idxmin()

  # Filter rows with minority class label
  minority_df = train_df[train_df.iloc[:, -1] == minority_class_label]

  # Calculate counts of majority and minority classes
  majority_count = class_counts.max()
  minority_count = class_counts.min()

  metadata_data = SingleTableMetadata()
  metadata_data.detect_from_dataframe(minority_df)
  # print(metadata_data)
  # Generate synthetic data using GaussianCopulaSynthesizer
  synthesizer_breast_data = GaussianCopulaSynthesizer(metadata_data)
  synthesizer_breast_data.fit(minority_df)

  # Print sample synthetic data
  synthesizer_breast_data.reset_sampling()
  sd1 = synthesizer_breast_data.sample(num_rows=majority_count-minority_count)
  return sd1, train_df

# Function to add synthetic data to the main DataFrame based on percentage
def add_synthetic_data(main_df, synthetic_df, percentage, seed=42):
    # Calculate number of rows to sample
    num_rows = int(len(synthetic_df) * percentage)
    np.random.seed(seed)
    # Sample the specified percentage of synthetic data
    sampled_synthetic_data = synthetic_df.sample(n=num_rows, replace=False, random_state=seed)
    # print(sampled_synthetic_data)

    # Concatenate sampled synthetic data with main DataFrame
    combined_df = pd.concat([main_df, sampled_synthetic_data], ignore_index=True)
    # print(combined_df)
    return combined_df

# Random Over-Sampling

In [112]:
def find_minority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    min_label = min(zip(counts, labels))[1]
    indices_with_min_label = np.where(y == min_label)[0]
    X_min, y_min = X[indices_with_min_label], y[indices_with_min_label]

    # Other class samples
    indices_without_min_label = np.where(y != min_label)[0]
    X_remaining, y_remaining = X[indices_without_min_label], y[indices_without_min_label]

    return X_min, y_min, X_remaining, y_remaining, min_label

def random_oversampling(X_train, y_train, oversampling_ratios, seed=42):

  oversampled_X_train_ratios = dict()
  oversampled_y_train_ratios = dict()
  X_minority, y_minority, X_remaining, y_remaining, min_label = find_minority_data(X_train, y_train)
  ideal_samps = len(X_remaining) - len(X_minority)

  oversampling_samps = [int(ideal_samps * (oversampling_ratio)) for oversampling_ratio in oversampling_ratios]
  for oversampling_samp, oversampling_ratio in zip(oversampling_samps, oversampling_ratios):

    sampling_strategy = {min_label: len(X_minority) + oversampling_samp}
    X_train_upsampled, y_train_upsampled = RandomOverSampler(sampling_strategy=sampling_strategy, random_state = seed).fit_resample(X_train, y_train)

    oversampled_X_train_ratios[oversampling_ratio] = X_train_upsampled
    oversampled_y_train_ratios[oversampling_ratio] = y_train_upsampled

  return list(oversampled_X_train_ratios.values()), list(oversampled_y_train_ratios.values())


# SMOTE Over-Sampling

In [113]:
def find_minority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    min_label = min(zip(counts, labels))[1]
    indices_with_min_label = np.where(y == min_label)[0]
    X_min, y_min = X[indices_with_min_label], y[indices_with_min_label]

    # Other class samples
    indices_without_min_label = np.where(y != min_label)[0]
    X_remaining, y_remaining = X[indices_without_min_label], y[indices_without_min_label]

    return X_min, y_min, X_remaining, y_remaining, min_label

def smote_oversampling(X_train, y_train, oversampling_ratios, seed=42):

  oversampled_X_train_ratios = dict()
  oversampled_y_train_ratios = dict()
  X_minority, y_minority, X_remaining, y_remaining, min_label = find_minority_data(X_train, y_train)
  ideal_samps = len(X_remaining) - len(X_minority)

  oversampling_samps = [int(ideal_samps * (oversampling_ratio)) for oversampling_ratio in oversampling_ratios]
  for oversampling_samp, oversampling_ratio in zip(oversampling_samps, oversampling_ratios):

    sampling_strategy = {min_label: len(X_minority) + oversampling_samp}
    X_train_upsampled, y_train_upsampled = SMOTE(sampling_strategy=sampling_strategy, random_state = seed).fit_resample(X_train, y_train)

    oversampled_X_train_ratios[oversampling_ratio] = X_train_upsampled
    oversampled_y_train_ratios[oversampling_ratio] = y_train_upsampled

  return list(oversampled_X_train_ratios.values()), list(oversampled_y_train_ratios.values())


# SVM-SMOTE Over-Sampling

In [114]:
def find_minority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    min_label = min(zip(counts, labels))[1]
    indices_with_min_label = np.where(y == min_label)[0]
    X_min, y_min = X[indices_with_min_label], y[indices_with_min_label]

    # Other class samples
    indices_without_min_label = np.where(y != min_label)[0]
    X_remaining, y_remaining = X[indices_without_min_label], y[indices_without_min_label]

    return X_min, y_min, X_remaining, y_remaining, min_label

def svm_smote_oversampling(X_train, y_train, oversampling_ratios, seed=42):

  oversampled_X_train_ratios = dict()
  oversampled_y_train_ratios = dict()
  X_minority, y_minority, X_remaining, y_remaining, min_label = find_minority_data(X_train, y_train)
  ideal_samps = len(X_remaining) - len(X_minority)

  oversampling_samps = [int(ideal_samps * (oversampling_ratio)) for oversampling_ratio in oversampling_ratios]
  for oversampling_samp, oversampling_ratio in zip(oversampling_samps, oversampling_ratios):

    sampling_strategy = {min_label: len(X_minority) + oversampling_samp}
    X_train_upsampled, y_train_upsampled = SVMSMOTE(sampling_strategy=sampling_strategy, random_state = seed).fit_resample(X_train, y_train)

    oversampled_X_train_ratios[oversampling_ratio] = X_train_upsampled
    oversampled_y_train_ratios[oversampling_ratio] = y_train_upsampled

  return list(oversampled_X_train_ratios.values()), list(oversampled_y_train_ratios.values())


# Intelligent Pruning

In [115]:
def find_majority_data(X, y):
    labels, counts = np.unique(y, return_counts=True)
    max_label = max(zip(counts, labels))[1]
    indices_with_max_label = np.where(y == max_label)[0]
    X_maj, y_maj = X[indices_with_max_label], y[indices_with_max_label]

    # Exclude majority class samples
    indices_without_max_label = np.where(y != max_label)[0]
    X_remaining, y_remaining = X[indices_without_max_label], y[indices_without_max_label]

    return X_maj, y_maj, X_remaining, y_remaining, min(counts)

def do_clustering(X, y, labels):
  clustered_X = defaultdict(list)
  clustered_y = defaultdict(list)

  for i, label in enumerate(labels):
      clustered_X[label].append(X[i])
      clustered_y[label].append(y[i])

  # Sort clustered_X and clustered_y in descending order based on the length of values in each dictionary
  sorted_clustered_X = dict(sorted(clustered_X.items(), key=lambda x: -len(x[1])))
  sorted_clustered_y = dict(sorted(clustered_y.items(), key=lambda x: -len(x[1])))

  return sorted_clustered_X, sorted_clustered_y


def intelligent_prune_data(pruning_samps, pruning_ratios, clustered_X, clustered_y, per_cluster_pruning_ratio=0.7, seed=42):
  random.seed(seed)
  pruning_ratios_X_maj, pruning_ratios_y_maj = defaultdict(list), defaultdict(list)
  for pruning_samp, pruning_ratio in zip(pruning_samps, pruning_ratios):
    samps = 0
    # print("For Pruning samps: ", pruning_samp)
    prune_samps = pruning_samp
    # print(prune_samps)
    clustered_X_new = defaultdict(list)
    clustered_y_new = defaultdict(list)
    # Iterate over the sorted dictionaries
    for label, values_X in clustered_X.items():
        # Calculate the number of samples to prune
        num_samples_to_prune = int(prune_samps * per_cluster_pruning_ratio)
        if(num_samples_to_prune > len(values_X)):
          num_samples_to_prune = len(values_X)//2
          prune_samps -= num_samples_to_prune
        else:
          prune_samps -= num_samples_to_prune

        # Randomly choose samples to prune
        indices_to_prune = random.sample(range(len(values_X)), num_samples_to_prune)

        # Prune the samples from clustered_X and clustered_y
        clustered_X_new[label] = [values_X[i] for i in range(len(values_X)) if i not in indices_to_prune]
        clustered_y_new[label] = [clustered_y[label][i] for i in range(len(clustered_y[label])) if i not in indices_to_prune]

    iter = 0
    while(prune_samps > 0):
        if(iter>=100):
          break
        for label, values_X in clustered_X_new.items():
          if(prune_samps <=0 or len(values_X) <= 0):
            break
          # print(len(values_X))
          index_to_prune = random.sample(range(len(values_X)), 1)
          clustered_X_new[label] = [values_X[i] for i in range(len(values_X)) if i not in index_to_prune]
          clustered_y_new[label] = [clustered_y_new[label][i] for i in range(len(clustered_y_new[label])) if i not in index_to_prune]

          prune_samps -= 1
        iter += 1

    for label in clustered_X_new:
        pruning_ratios_X_maj[pruning_ratio].extend(clustered_X_new[label])
        pruning_ratios_y_maj[pruning_ratio].extend(clustered_y_new[label])

  return pruning_ratios_X_maj, pruning_ratios_y_maj

def combine_data(pruning_ratios, pruning_ratios_X_maj, pruning_ratios_y_maj, X_remaining, y_remaining):

  pruning_ratios_X, pruning_ratios_y = defaultdict(list), defaultdict(list)
  for pruning_ratio in pruning_ratios:
    pruning_ratios_X[pruning_ratio].extend(pruning_ratios_X_maj[pruning_ratio])
    pruning_ratios_X[pruning_ratio].extend(X_remaining)

    pruning_ratios_y[pruning_ratio].extend(pruning_ratios_y_maj[pruning_ratio])
    pruning_ratios_y[pruning_ratio].extend(y_remaining)

  return pruning_ratios_X, pruning_ratios_y

def do_intelligent_pruning(X, y, ratio, per_cluster_pruning_ratio=0.7, seed=42):

  X_maj, y_maj, X_remaining, y_remaining, min_class_samples = find_majority_data(X, y)
  kmeans = KMeans(n_clusters=3, random_state = 42)
  kmeans.fit(X_maj)
  labels = kmeans.labels_
  clustered_X, clustered_y = do_clustering(X_maj, y_maj, labels)

  pruning_best = len(X_maj)-min_class_samples
  pruning_samps = [int(pruning_best * ratio)]
  pruning_ratios = [ratio]

  pruning_ratios_X_maj, pruning_ratios_y_maj = intelligent_prune_data(pruning_samps, pruning_ratios, clustered_X, clustered_y, \
                                                                      per_cluster_pruning_ratio=per_cluster_pruning_ratio, seed=seed)

  pruning_ratios_X, pruning_ratios_y = combine_data(pruning_ratios, pruning_ratios_X_maj, pruning_ratios_y_maj, X_remaining, y_remaining)

  return list(pruning_ratios_X.values()), list(pruning_ratios_y.values())

# Random Pruning

In [116]:
"""
inputs:
X: np.array
y: np.array
percentage: from 0.0 upto 1.0, enter int value
"""
def random_prune_data(X, y, ratio, seed = 42):
  # preprocessed_X, scaler, imputer = preprocess_data_train(X)
  # preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

  # X_train, y_train = preprocessed_X_train.to_numpy(), y_train.to_numpy()
  # X_test, y_test = preprocessed_X_test.to_numpy(), y_test.to_numpy()
  np.random.seed(seed)
  labels_count = {}
  labels = np.unique(y)
  for label in labels:
    labels_count[label] = np.count_nonzero(y == label)
  max_label = min_label = labels[0]
  for label in labels_count:
    if labels_count[label] > labels_count[max_label]:
      max_label = label
    if labels_count[label] < labels_count[min_label]:
      min_label = label

  # print("Max", max_label, labels_count[max_label])
  # print("Min", min_label, labels_count[min_label])

  prune_counts = {}
  prune_indexes = {}
  for label in labels_count:
    prune_counts[label] = labels_count[label] - labels_count[min_label]
    prune_indexes[label] = np.where(y == label)[0]

  prune_amount = int(ratio * sum(map(lambda x: x[1], prune_counts.items())))
  prune_it = {}

  while prune_amount > 0:
    for label in labels:
      if (len(prune_indexes[label]) - labels_count[min_label]) > 0 and prune_amount > 0:
        random_index = np.random.choice(len(prune_indexes[label]))
        random_item = prune_indexes[label][random_index]
        prune_indexes[label] = np.delete(prune_indexes[label], random_index)
        if prune_it.get(label, None) is None:
          prune_it[label] = np.array([])
        prune_it[label] = np.append(prune_it[label], [random_item])
        prune_amount -= 1



  formatted_indexes = np.array([])
  for label in prune_indexes:
    formatted_indexes = np.append(formatted_indexes, prune_indexes[label])
  formatted_indexes = np.sort(formatted_indexes)
  new_arr = np.array([np.int64(i) for i in formatted_indexes])

  return X[new_arr], y[new_arr]

## SMO Analysis

In [117]:
def apply_smo(X, y, smo_K, random_state, budget=10):

    # Get indices for each class
    np.random.seed(random_state)
    class_indices = [np.where(y == i)[0] for i in np.unique(y)]
    # Shuffle class_indices
    for indices in class_indices:
        np.random.shuffle(indices)

    # Select K samples for each class
    selected_indices = [indices[:smo_K] for indices in class_indices]

    # Flatten the list of indices
    selected_indices_flat = np.concatenate(selected_indices)

    # Get the selected data and target values
    selected_data = X.loc[selected_indices_flat]
    selected_target = y.loc[selected_indices_flat]

    selected_data_df = pd.DataFrame(selected_data, columns=X.columns)
    selected_target_df = pd.Series(selected_target, name=y.name)

    # Store the remaining samples as well
    remaining_indices_flat = np.setdiff1d(np.arange(len(X)), selected_indices_flat)
    remaining_data = X.loc[remaining_indices_flat]
    remaining_target =  y.loc[remaining_indices_flat]

    clf = GaussianNB()
    clf.fit(selected_data, selected_target)

    for i in range(budget):
        out, thresh, out_label = 0, 1E-30, 0
        for index, row in remaining_data.iterrows():
            target_value = remaining_target.loc[index]
            row_2d = row.values.reshape(1, -1)
            p1, p2 = clf.predict_proba(row_2d)[0][0], clf.predict_proba(row_2d)[0][1]
            score = abs(p1+p2)/(abs(p1-p2)+1E-300)
            if(score > thresh):
                out = index
                thresh = score
                if(p1<p2):
                    out_label = 1
                else:
                    out_label = 0

        X_to_move = (remaining_data.loc[out]).values.reshape(1, -1)
        y_to_move = [out_label]

        # Append the row to remaining_data
        X_to_move_df = pd.DataFrame(X_to_move, columns=remaining_data.columns)
        selected_data_df = pd.concat([selected_data_df, X_to_move_df], ignore_index=True)

        # Append the value to remaining_target
        y_to_move_series = pd.Series(y_to_move, name=remaining_target.name)
        selected_target_df = pd.concat([selected_target_df, y_to_move_series], ignore_index=True)

        clf.partial_fit(X_to_move, y_to_move)

        remaining_data = remaining_data.drop(out).reset_index(drop=True)
        remaining_target = remaining_target.drop(out).reset_index(drop=True)


    remaining_data_df = pd.DataFrame(remaining_data, columns=X.columns)

    remaining_target_df = pd.Series(clf.predict(remaining_data_df), name=remaining_target.name)
    matches = (remaining_target_df == remaining_target).sum()
    X_new = pd.concat([selected_data_df, remaining_data_df])
    y_new = pd.concat([selected_target_df, remaining_target_df])

    return X_new, y_new, matches, len(remaining_target_df)

In [118]:
ratios = [ratio for ratio in np.arange(0, 1.1, 0.2)]

In [119]:
learners = {'Logistic Regression' : 'LR', 'SVM': 'SVM', 'Decision Tree': 'DT' }

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

states = [82, 15, 4, 95, 36, 32, 29, 18, 14, 87]
ratios = [0.2, 0.4, 0.6, 0.8, 1.0]
smo_K = 30
smo = True

In [120]:
def split_data(X, y, random_state):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_state)
  return X_train, X_test, y_train, y_test

In [121]:
smo_X_train, smo_y_train = dict(), dict()
for rand_state in states:
  X_train, X_test, y_train, y_test = split_data(X, y,rand_state)
  X_train.reset_index(drop=True, inplace=True)
  X_test.reset_index(drop=True, inplace=True)
  y_train.reset_index(drop=True, inplace=True)
  y_test.reset_index(drop=True, inplace=True)
  X_train, y_train, matches, total = apply_smo(X_train, y_train, smo_K, rand_state)

  smo_X_train[rand_state] = X_train
  smo_y_train[rand_state] = y_train
  print(str(matches) + " Matches out of " + str(total) + " dark samples")

605 Matches out of 727 dark samples
582 Matches out of 727 dark samples
592 Matches out of 727 dark samples
596 Matches out of 727 dark samples
612 Matches out of 727 dark samples
609 Matches out of 727 dark samples
619 Matches out of 727 dark samples
560 Matches out of 727 dark samples
589 Matches out of 727 dark samples
611 Matches out of 727 dark samples


# Calling Intelligent Pruning

In [122]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'intelligent_pruning'
file_name = f'{algorithm}_results'
per_cluster_pruning_ratios = [0.5, 0.7, 0.9, 1]

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]

  for per_cluster_pruning_ratio in per_cluster_pruning_ratios:
    for ratio in ratios:
      X_train_copy, y_train_copy = X_train.copy(), y_train.copy()

      intelligent_pruned_X_train, intelligent_pruned_y_train = do_intelligent_pruning(X_train_copy.to_numpy(), y_train_copy.to_numpy(), ratio, per_cluster_pruning_ratio=per_cluster_pruning_ratio)

      preprocessed_intelligent_pruned_X_train, scaler, imputer = preprocess_data_train((np.array(intelligent_pruned_X_train))[0])
      preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

      intelligent_pruned_X_train, intelligent_pruned_y_train = preprocessed_intelligent_pruned_X_train, (np.array(intelligent_pruned_y_train))[0]
      intelligent_pruned_X_test, intelligent_pruned_y_test = preprocessed_X_test, y_test.to_numpy()
      results = evaluate_models(intelligent_pruned_X_train, intelligent_pruned_X_test, intelligent_pruned_y_train, intelligent_pruned_y_test, rand_state)
      print(results)
      for key, item in results.items():
        row = {'algo' : [algorithm], 'ratio': [ratio], 'per_cluster_pruning_ratio': [per_cluster_pruning_ratio],'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
        temp = pd.DataFrame(row)
        df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)


{'Logistic Regression': {'Accuracy': 0.785, 'Precision': 0.5510204081632653, 'Recall': 0.5625, 'F1 Score': 0.5567010309278351, 'ROC AUC': 0.7521929824561402, 'Confusion Matrix': array([[130,  22],
       [ 21,  27]])}, 'SVM': {'Accuracy': 0.785, 'Precision': 0.5531914893617021, 'Recall': 0.5416666666666666, 'F1 Score': 0.5473684210526316, 'ROC AUC': 0.7043585526315789, 'Confusion Matrix': array([[131,  21],
       [ 22,  26]])}, 'Decision Tree': {'Accuracy': 0.79, 'Precision': 0.5555555555555556, 'Recall': 0.625, 'F1 Score': 0.5882352941176471, 'ROC AUC': 0.7335526315789472, 'Confusion Matrix': array([[128,  24],
       [ 18,  30]])}}
{'Logistic Regression': {'Accuracy': 0.78, 'Precision': 0.5384615384615384, 'Recall': 0.5833333333333334, 'F1 Score': 0.5599999999999999, 'ROC AUC': 0.7482182017543859, 'Confusion Matrix': array([[128,  24],
       [ 20,  28]])}, 'SVM': {'Accuracy': 0.775, 'Precision': 0.5294117647058824, 'Recall': 0.5625, 'F1 Score': 0.5454545454545455, 'ROC AUC': 0.7098

#Calling Random Pruning

In [123]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'random_pruning'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]

  for ratio in ratios:

    random_pruned_X_train, random_pruned_y_train = random_prune_data(X_train.to_numpy(), y_train.to_numpy(), ratio, seed=rand_state)
    preprocessed_random_pruned_X_train, scaler, imputer = preprocess_data_train(random_pruned_X_train)
    preprocessed_X_test = preprocess_data_test(X_test, scaler, imputer)

    random_pruned_X_train, random_pruned_y_train = preprocessed_random_pruned_X_train, random_pruned_y_train
    random_pruned_X_test, random_pruned_y_test = preprocessed_X_test, y_test.to_numpy()

    print(f"Train data pruned randomly at {ratio * 100}%")
    results = evaluate_models(random_pruned_X_train, random_pruned_X_test, random_pruned_y_train, random_pruned_y_test, rand_state)
    print(results)
    for key, item in results.items():
      row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
      temp = pd.DataFrame(row)
      df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)


Train data pruned randomly at 20.0%
{'Logistic Regression': {'Accuracy': 0.77, 'Precision': 0.5208333333333334, 'Recall': 0.5208333333333334, 'F1 Score': 0.5208333333333334, 'ROC AUC': 0.7515076754385964, 'Confusion Matrix': array([[129,  23],
       [ 23,  25]])}, 'SVM': {'Accuracy': 0.78, 'Precision': 0.5454545454545454, 'Recall': 0.5, 'F1 Score': 0.5217391304347826, 'ROC AUC': 0.7077850877192983, 'Confusion Matrix': array([[132,  20],
       [ 24,  24]])}, 'Decision Tree': {'Accuracy': 0.765, 'Precision': 0.5094339622641509, 'Recall': 0.5625, 'F1 Score': 0.5346534653465346, 'ROC AUC': 0.6957236842105263, 'Confusion Matrix': array([[126,  26],
       [ 21,  27]])}}
Train data pruned randomly at 40.0%
{'Logistic Regression': {'Accuracy': 0.775, 'Precision': 0.5306122448979592, 'Recall': 0.5416666666666666, 'F1 Score': 0.5360824742268041, 'ROC AUC': 0.7495888157894738, 'Confusion Matrix': array([[129,  23],
       [ 22,  26]])}, 'SVM': {'Accuracy': 0.77, 'Precision': 0.5208333333333334

# Calling SDV-Oversampling

In [124]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'gaussian_copula'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]

  for ratio in ratios:

    sd1, train_df = do_sdv(X_train, y_train)
    results_syn_sdv = dict()

    # Add synthetic data at different percentages to the main DataFrame
    combined_df = add_synthetic_data(train_df, sd1, ratio, seed=rand_state)
    y_train_sdv = combined_df.iloc[:,-1]
    X_train_sdv = combined_df.iloc[:,:-1]
    try:
      preprocessed_X_train_sdv, scaler, imputer = preprocess_data_train(X_train_sdv)
      preprocessed_X_test_sdv = preprocess_data_test(X_test, scaler, imputer)
      X_train_sdv, y_train_sdv = preprocessed_X_train_sdv, y_train_sdv.to_numpy()
      X_test_sdv, y_test_sdv = preprocessed_X_test_sdv, y_test.to_numpy()

      print(f"Train data combined with {ratio * 100}% synthetic data of minority class")
      results = evaluate_models(X_train_sdv, X_test_sdv, y_train_sdv, y_test_sdv, rand_state)
      print(results)
      for key, item in results.items():
            row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
            temp = pd.DataFrame(row)
            df = pd.concat([df, temp], ignore_index=True)
    except:
      continue
df.to_csv(file_name, index=False)

Train data combined with 20.0% synthetic data of minority class
{'Logistic Regression': {'Accuracy': 0.775, 'Precision': 0.5319148936170213, 'Recall': 0.5208333333333334, 'F1 Score': 0.5263157894736842, 'ROC AUC': 0.755345394736842, 'Confusion Matrix': array([[130,  22],
       [ 23,  25]])}, 'SVM': {'Accuracy': 0.78, 'Precision': 0.5434782608695652, 'Recall': 0.5208333333333334, 'F1 Score': 0.5319148936170214, 'ROC AUC': 0.7131304824561403, 'Confusion Matrix': array([[131,  21],
       [ 23,  25]])}, 'Decision Tree': {'Accuracy': 0.77, 'Precision': 0.5185185185185185, 'Recall': 0.5833333333333334, 'F1 Score': 0.5490196078431373, 'ROC AUC': 0.7061403508771931, 'Confusion Matrix': array([[126,  26],
       [ 20,  28]])}}
Train data combined with 40.0% synthetic data of minority class
{'Logistic Regression': {'Accuracy': 0.775, 'Precision': 0.5319148936170213, 'Recall': 0.5208333333333334, 'F1 Score': 0.5263157894736842, 'ROC AUC': 0.754797149122807, 'Confusion Matrix': array([[130,  22]

# Calling SMOTE-Oversampling

In [125]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'smote'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]

  for ratio in ratios:
    X_train_smote, y_train_smote = smote_oversampling(X_train.to_numpy(), y_train.to_numpy(), [ratio])
    preprocessed_X_train_smote, scaler, imputer = preprocess_data_train((np.array(X_train_smote))[0])
    preprocessed_X_test_smote = preprocess_data_test(X_test, scaler, imputer)

    X_train_smote, y_train_smote = preprocessed_X_train_smote, (np.array(y_train_smote))[0]
    X_test_smote, y_test_smote = preprocessed_X_test_smote, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    results = evaluate_models(X_train_smote, X_test_smote, y_train_smote, y_test_smote, rand_state)
    print(results)

    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

Train data combined with 20.0% synthetic data of minority class:
{'Logistic Regression': {'Accuracy': 0.78, 'Precision': 0.5416666666666666, 'Recall': 0.5416666666666666, 'F1 Score': 0.5416666666666666, 'ROC AUC': 0.7578125, 'Confusion Matrix': array([[130,  22],
       [ 22,  26]])}, 'SVM': {'Accuracy': 0.765, 'Precision': 0.5102040816326531, 'Recall': 0.5208333333333334, 'F1 Score': 0.5154639175257733, 'ROC AUC': 0.7124451754385964, 'Confusion Matrix': array([[128,  24],
       [ 23,  25]])}, 'Decision Tree': {'Accuracy': 0.775, 'Precision': 0.5283018867924528, 'Recall': 0.5833333333333334, 'F1 Score': 0.5544554455445545, 'ROC AUC': 0.7094298245614036, 'Confusion Matrix': array([[127,  25],
       [ 20,  28]])}}
Train data combined with 40.0% synthetic data of minority class:
{'Logistic Regression': {'Accuracy': 0.77, 'Precision': 0.5192307692307693, 'Recall': 0.5625, 'F1 Score': 0.54, 'ROC AUC': 0.7613760964912281, 'Confusion Matrix': array([[127,  25],
       [ 21,  27]])}, 'SVM': 

# Calling Random-Oversampling

In [126]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'random_oversampling'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for ratio in ratios:
    X_train_random, y_train_random = random_oversampling(X_train.to_numpy(), y_train.to_numpy(), [ratio])

    preprocessed_X_train_random, scaler, imputer = preprocess_data_train((np.array(X_train_random)[0]))
    preprocessed_X_test_random = preprocess_data_test(X_test, scaler, imputer)

    X_train_random, y_train_random = preprocessed_X_train_random, (np.array(y_train_random))[0]
    X_test_random, y_test_random = preprocessed_X_test_random, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    results = evaluate_models(X_train_random, X_test_random, y_train_random, y_test_random, rand_state)
    print(results)

    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

Train data combined with 20.0% synthetic data of minority class:
{'Logistic Regression': {'Accuracy': 0.775, 'Precision': 0.5306122448979592, 'Recall': 0.5416666666666666, 'F1 Score': 0.5360824742268041, 'ROC AUC': 0.7479440789473685, 'Confusion Matrix': array([[129,  23],
       [ 22,  26]])}, 'SVM': {'Accuracy': 0.77, 'Precision': 0.5208333333333334, 'Recall': 0.5208333333333334, 'F1 Score': 0.5208333333333334, 'ROC AUC': 0.7224506578947368, 'Confusion Matrix': array([[129,  23],
       [ 23,  25]])}, 'Decision Tree': {'Accuracy': 0.78, 'Precision': 0.54, 'Recall': 0.5625, 'F1 Score': 0.5510204081632654, 'ROC AUC': 0.7055921052631579, 'Confusion Matrix': array([[129,  23],
       [ 21,  27]])}}
Train data combined with 40.0% synthetic data of minority class:
{'Logistic Regression': {'Accuracy': 0.78, 'Precision': 0.54, 'Recall': 0.5625, 'F1 Score': 0.5510204081632654, 'ROC AUC': 0.7524671052631579, 'Confusion Matrix': array([[129,  23],
       [ 21,  27]])}, 'SVM': {'Accuracy': 0.77,

# Calling SVM-SMOTE Over-Sampling

In [127]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'svm_smote'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]
  for ratio in ratios:
    X_train_svm_smote, y_train_svm_smote = svm_smote_oversampling(X_train.to_numpy(), y_train.to_numpy(), [ratio])

    preprocessed_X_train_svm_smote, scaler, imputer = preprocess_data_train((np.array(X_train_svm_smote))[0])
    preprocessed_X_test_svm_smote = preprocess_data_test(X_test, scaler, imputer)

    X_train_svm_smote, y_train_svm_smote = preprocessed_X_train_svm_smote, (np.array(y_train_svm_smote))[0]
    X_test_svm_smote, y_test_svm_smote = preprocessed_X_test_svm_smote, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class:")
    results = evaluate_models(X_train_svm_smote, X_test_svm_smote, y_train_svm_smote, y_test_svm_smote, rand_state)
    print(results)

    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

Train data combined with 20.0% synthetic data of minority class:
{'Logistic Regression': {'Accuracy': 0.775, 'Precision': 0.5306122448979592, 'Recall': 0.5416666666666666, 'F1 Score': 0.5360824742268041, 'ROC AUC': 0.7436266447368421, 'Confusion Matrix': array([[129,  23],
       [ 22,  26]])}, 'SVM': {'Accuracy': 0.77, 'Precision': 0.5192307692307693, 'Recall': 0.5625, 'F1 Score': 0.54, 'ROC AUC': 0.7210800438596492, 'Confusion Matrix': array([[127,  25],
       [ 21,  27]])}, 'Decision Tree': {'Accuracy': 0.765, 'Precision': 0.5094339622641509, 'Recall': 0.5625, 'F1 Score': 0.5346534653465346, 'ROC AUC': 0.6957236842105263, 'Confusion Matrix': array([[126,  26],
       [ 21,  27]])}}
Train data combined with 40.0% synthetic data of minority class:
{'Logistic Regression': {'Accuracy': 0.77, 'Precision': 0.5192307692307693, 'Recall': 0.5625, 'F1 Score': 0.54, 'ROC AUC': 0.7567845394736842, 'Confusion Matrix': array([[127,  25],
       [ 21,  27]])}, 'SVM': {'Accuracy': 0.765, 'Precisio

# Calling for RRP Over-Sampling

In [128]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'RRP'
file_name = f'{algorithm}_results'

for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]

  for ratio in ratios:
    X_train_copy, y_train_copy = X_train.copy(), y_train.copy()

    rt, new_data_df, train_df = RandomProjectionOversampling(X_train=X_train_copy,
                                                                        y_train=y_train_copy,
                                                                        threshold=10)

    # Add synthetic data at different percentages to the main DataFrame
    combined_df = add_synthetic_data(train_df, new_data_df, ratio, seed=rand_state)
    y_train_rrp = combined_df.iloc[:,-1]
    X_train_rrp = combined_df.iloc[:,:-1]

    preprocessed_X_train_rrp, scaler, imputer = preprocess_data_train(X_train_rrp)
    preprocessed_X_test_rrp = preprocess_data_test(X_test, scaler, imputer)

    X_train_rrp, y_train_rrp = preprocessed_X_train_rrp, y_train_rrp.to_numpy()
    X_test_rrp, y_test_rrp = preprocessed_X_test_rrp, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class")
    results = evaluate_models(X_train_rrp, X_test_rrp, y_train_rrp, y_test_rrp, rand_state)
    print(results)

    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

188 53 467 414
Train data combined with 20.0% synthetic data of minority class
{'Logistic Regression': {'Accuracy': 0.78, 'Precision': 0.5454545454545454, 'Recall': 0.5, 'F1 Score': 0.5217391304347826, 'ROC AUC': 0.748766447368421, 'Confusion Matrix': array([[132,  20],
       [ 24,  24]])}, 'SVM': {'Accuracy': 0.775, 'Precision': 0.5333333333333333, 'Recall': 0.5, 'F1 Score': 0.5161290322580646, 'ROC AUC': 0.7135416666666666, 'Confusion Matrix': array([[131,  21],
       [ 24,  24]])}, 'Decision Tree': {'Accuracy': 0.78, 'Precision': 0.5384615384615384, 'Recall': 0.5833333333333334, 'F1 Score': 0.5599999999999999, 'ROC AUC': 0.7127192982456141, 'Confusion Matrix': array([[128,  24],
       [ 20,  28]])}}
182 88 467 379
Train data combined with 40.0% synthetic data of minority class
{'Logistic Regression': {'Accuracy': 0.78, 'Precision': 0.54, 'Recall': 0.5625, 'F1 Score': 0.5510204081632654, 'ROC AUC': 0.7584978070175438, 'Confusion Matrix': array([[129,  23],
       [ 21,  27]])}, 'S

# No-Sampling

In [129]:
cols = ['algo', 'ratio', 'seed', 'learner', 'acc', 'prec', 'rec', 'f1', 'auc_roc']
df = pd.DataFrame([], columns = cols)
algorithm = 'No_Sampling'
file_name = f'{algorithm}_results'

ratios = [0]
for rand_state in states:
  if(smo):

    X_train, y_train = smo_X_train[rand_state], smo_y_train[rand_state]

  for ratio in ratios:

    X_train_no_sampling, y_train_no_sampling = X_train.copy(), y_train.copy()

    preprocessed_X_train_no_sampling, scaler, imputer = preprocess_data_train(X_train_no_sampling)
    preprocessed_X_test_no_sampling = preprocess_data_test(X_test, scaler, imputer)

    X_train_no_sampling, y_train_no_sampling = preprocessed_X_train_no_sampling, y_train_no_sampling.to_numpy()
    X_test_no_sampling, y_test_no_sampling = preprocessed_X_test_no_sampling, y_test.to_numpy()

    print(f"Train data combined with {ratio * 100}% synthetic data of minority class")
    results = evaluate_models(X_train_no_sampling, X_test_no_sampling, y_train_no_sampling, y_test_no_sampling, rand_state)
    print(results)

    for key, item in results.items():
          row = {'algo' : [algorithm], 'ratio': [ratio], 'seed': [rand_state], 'learner': [learners[key]], 'acc': [item['Accuracy']], 'prec': [item['Precision']], 'rec': [item['Recall']], 'f1': [item['F1 Score']], 'auc_roc': [item['ROC AUC']]}
          temp = pd.DataFrame(row)
          df = pd.concat([df, temp], ignore_index=True)
df.to_csv(file_name, index=False)

Train data combined with 0% synthetic data of minority class
{'Logistic Regression': {'Accuracy': 0.775, 'Precision': 0.5319148936170213, 'Recall': 0.5208333333333334, 'F1 Score': 0.5263157894736842, 'ROC AUC': 0.7534265350877193, 'Confusion Matrix': array([[130,  22],
       [ 23,  25]])}, 'SVM': {'Accuracy': 0.785, 'Precision': 0.5581395348837209, 'Recall': 0.5, 'F1 Score': 0.5274725274725274, 'ROC AUC': 0.707236842105263, 'Confusion Matrix': array([[133,  19],
       [ 24,  24]])}, 'Decision Tree': {'Accuracy': 0.78, 'Precision': 0.5384615384615384, 'Recall': 0.5833333333333334, 'F1 Score': 0.5599999999999999, 'ROC AUC': 0.7127192982456141, 'Confusion Matrix': array([[128,  24],
       [ 20,  28]])}}
Train data combined with 0% synthetic data of minority class
{'Logistic Regression': {'Accuracy': 0.775, 'Precision': 0.5306122448979592, 'Recall': 0.5416666666666666, 'F1 Score': 0.5360824742268041, 'ROC AUC': 0.7384868421052632, 'Confusion Matrix': array([[129,  23],
       [ 22,  26]